In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy.integrate import odeint

%matplotlib notebook

In [ ]:
def SMM(i, j, N, epsilon, pi):
    return epsilon * (i == j) + (1 - epsilon) * pi[j] * N[j] / np.sum(pi * N)


def dSidt(S, Ires, Isen, N, pi, i, alpha, beta, gamma, epsilon, phi, mu, D):
    nu = (1 - phi) / D
    tau = phi / D

    dSi = -S[i] * pi[i] * sum([
        SMM(i, j, N, epsilon, pi) * beta[i, j] * (Isen[j] + Ires[j]) / N[j]
        for j in [0, 1]
    ]) + nu * (Isen[i] + Ires[i]) + tau * (1 - mu) * Isen[i] + alpha * (
        -S[i] + N[i]) - gamma * S[i] + gamma * N[i] * sum(S)

    return dSi


def dIsenidt(S, Ires, Isen, N, pi, i, alpha, beta, gamma, epsilon, phi, mu, D):
    nu = (1 - phi) / D
    tau = phi / D

    dIseni = S[i] * pi[i] * sum([
        SMM(i, j, N, epsilon, pi) * beta[i, j] * Isen[j] / N[j]
        for j in [0, 1]
    ]) - (nu + tau + alpha + gamma) * Isen[i] + gamma * N[i] * np.sum(Isen)

    return dIseni


def dIresidt(S, Ires, Isen, N, pi, i, alpha, beta, gamma, epsilon, phi, mu, D):
    nu = (1 - phi) / D
    tau = phi / D

    dIresi = S[i] * pi[i] * sum([
        SMM(i, j, N, epsilon, pi) * beta[i, j] * Ires[j] / N[j]
        for j in [0, 1]
    ]) - (nu + alpha + gamma) * Ires[i] + tau * mu * Isen[i] + gamma * N[i] * np.sum(Ires)

    return dIresi


def model(y, t, N, pi, alpha, beta, gamma, epsilon, phi, mu, D):
    dydt = []
    S = y[:2]
    Ires = y[2:4]
    Isen = y[4:]

    funcs = [dSidt, dIresidt, dIsenidt]

    for func in funcs:
        for i in range(2):
            dydt.append(
                func(S, Ires, Isen, N, pi, i, alpha, beta, gamma, epsilon, phi,
                     mu, D))

    return dydt

MSM

In [ ]:
Nh = 5.3e-2
N = np.array([1 - Nh, Nh])

S = np.array([N[0]*0.995,N[1]*0.90])
Ires = np.array([0, 0])
Isen = np.array([N[0]*0.005, N[1]*0.1])

pi = np.array([0.41, 30.49])


BLL = np.array([0.42,0.59,0.77])
BHH = np.array([0.25,0.30,0.40])
alpha = 1 / 29


gamma = 1
epsilon = np.array([0.30,0.57,0.80])
phi = np.array([0.48,0.64,0.81])
mu = 1e-3
D = np.array([0.14,0.19,0.25])

y0 = np.concatenate((S, Ires, Isen))
# y0 /= np.sum(y0)
t = np.arange(0,30,0.001)
sol_msm = []
for i in range(3):
    beta = np.array([[BLL[i], 1], [1, BHH[i]]])
    beta[0, 1] = beta[1, 0] = np.sqrt(beta[0, 0] * beta[1, 1])
    sol_msm.append(odeint(model, y0, t, args=(N, pi, alpha, beta, gamma, epsilon[i], phi[i], mu, D[i])))
    
sol_msm = np.array(sol_msm)

HMW

In [ ]:
Nh = 6.3e-2
N = np.array([1 - Nh, Nh])

S = np.array([N[0]*0.995,N[1]*0.90])
Ires = np.array([0, 0])
Isen = np.array([N[0]*0.005, N[1]*0.1])

pi = np.array([0.25, 4.57])


alpha = 1 / 29
BLL = np.array([0.79,0.87,0.94])
BHH = np.array([0.63,0.72,0.81])

gamma = 1
epsilon = np.array([0.53,0.73,0.89])
phi = np.array([0.36,0.50,0.66])
mu = 1e-3
D = np.array([0.46,0.55,0.66])

y0 = np.concatenate((S, Ires, Isen))
# y0 /= np.sum(y0)
t = np.arange(0,30,0.001)
sol_hmw = []
for i in range(3):
    beta = np.array([[BLL[i], 1], [1, BHH[i]]])
    beta[0, 1] = beta[1, 0] = np.sqrt(beta[0, 0] * beta[1, 1])
    sol_hmw.append(odeint(model, y0, t, args=(N, pi, alpha, beta, gamma, epsilon[i], phi[i], mu, D[i])))
    
sol_hmw = np.array(sol_hmw)

In [ ]:
plt.figure()
plt.plot(t, sol_msm[:, 0], label='Sl')
plt.plot(t, sol_msm[:, 1], label='Sh')
plt.plot(t, sol_msm[:, 2], label='Iresl')
plt.plot(t, sol_msm[:, 3], label='Iresh')
plt.plot(t, sol_msm[:, 4], label='Isenl')
plt.plot(t, sol_msm[:, 5], label='Isenh')

plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.show()



In [ ]:
plt.figure()

ratio_sol_msm = (sol_msm[:,:, 2] + sol_msm[:,:, 3])/(sol_msm[:,:, 2] + sol_msm[:,:, 3] + sol_msm[:,:, 4] + sol_msm[:,:, 5])
plt.plot(t, ratio_sol_msm[1,:], label = "MSM", c = "g",linewidth=1)
plt.fill_between(t,ratio_sol_msm[0,:], ratio_sol_msm[2,:], alpha=0.5, facecolor = "g")

ratio_sol_hmw = (sol_hmw[:,:, 2] + sol_hmw[:,:, 3])/(sol_hmw[:,:, 2] + sol_hmw[:,:, 3] + sol_hmw[:,:, 4] + sol_hmw[:,:, 5])
plt.plot(t, ratio_sol_hmw[1,:], label = "HMW", c = "b",linewidth=1)
plt.fill_between(t,ratio_sol_hmw[0,:], ratio_sol_hmw[2,:], alpha=0.5, facecolor = "b")



plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.ylim(0,1)
plt.show()